In [8]:
from flask import Flask, render_template, request
from gtts import gTTS
import io

@app.route('/')
def home():
    return "Welcome to the Text-to-Speech App!"

@app.route('/read_text', methods=['GET', 'POST'])
def read_text():
    if request.method == 'POST':
        text = request.form.get('text', '')

        if text:
            # Create a gTTS object to convert text to speech
            tts = gTTS(text)
            
            # Save the speech as an in-memory file
            speech_io = io.BytesIO()
            tts.write_to_fp(speech_io)
            speech_io.seek(0)

            return render_template('read_text.html', speech=speech_io.getvalue())

    return render_template('read_text.html', speech=None)



/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [6]:
%%writefile templates/read_text.html
<!DOCTYPE html>
<html>
<head>
    <title>Text-to-Speech</title>
</head>
<body>
    <h1>Text-to-Speech</h1>
    <form method="POST">
        <textarea name="text" rows="20" cols="80" placeholder="Enter text"></textarea><br>
        <button type="submit">Read Aloud</button>
    </form>

    {% if speech %}
    <audio controls>
        <source src="data:audio/mpeg;base64,{{ speech|b64encode }}" type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
    {% endif %}
</body>
</html>


Writing templates/read_text.html


In [7]:
from flask import Flask, render_template
import sqlite3
from datetime import datetime
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')
from flask import request, render_template

@app.route('/searchdb', methods=['GET', 'POST'])
def searchdb():
    if request.method == 'POST':
        DATA = []
        ROWID = None  # Initialize ROWID

        search_term = request.form.get('search_input', '')  # Provide a default value
        conn = sqlite3.connect("notes.db")
        conn.text_factory = str
        c = conn.cursor()
        
        for row in c.execute("SELECT ROWID,* FROM PROJECT"):
            if search_term in row[1]:
                DATA.append("\nINFO Found in ROWID: " + str(row[0]) + "\n" + row[1])
                ROWID = row[0]  # Update ROWID
        
        return render_template('searchdb.html', DATA=DATA, ROWID=ROWID)
    
    return render_template('search_form.html')


@app.route('/show_input_form')
def show_input_form():
    return render_template('input_form.html')

@app.route('/insert', methods=['POST'])
def insert_into_db():
    data = request.form.get('data')
    if data:
        conn = sqlite3.connect("notes.db")
        conn.text_factory = str
        c = conn.cursor()
        
        timestamp = datetime.now().strftime("%A %x %H:%M:%S")
        data_with_timestamp = data + "\n" + timestamp
        
        c.execute("INSERT INTO PROJECT (input) VALUES (?)", (data_with_timestamp,))
        conn.commit()
        inserted_row_id = c.lastrowid  # Get the ID of the last inserted row
        
        # Retrieve the inserted row using its ID
        c.execute("SELECT * FROM PROJECT WHERE ROWID = ?", (inserted_row_id,))
        inserted_row = c.fetchone()
        
        conn.close()
        
        # Format the inserted data for display within <pre> tag
        formatted_data = "\n".join(inserted_row[0].split("\n"))
        
        return f"Data inserted successfully!\nInserted Row: <pre>{formatted_data}</pre>"
    else:
        return "No data provided."

    
    
@app.route('/delete/<int:row_id>', methods=['GET', 'POST'])
def delete_entry(row_id):
    conn = sqlite3.connect("notes.db")
    c = conn.cursor()

    # Retrieve the data associated with the row_id
    c.execute("SELECT ROWID, * FROM PROJECT WHERE ROWID = ?", (row_id,))

    entry = c.fetchone()

    if entry:
        # Delete the entry based on the row_id
        c.execute("DELETE FROM PROJECT WHERE ROWID = ?", (row_id,))
        conn.commit()
        conn.close()

        return f"Entry with ROWID {row_id} deleted successfully!"
    else:
        conn.close()
        return f"No entry found with ROWID {row_id}."

if __name__ == '__main__':
    app.run(debug=True, port=5001)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 689, in initialize
    self.init_sockets()
  File "/home/jack/Desktop/YOUTUBE_video/flask_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 328, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/jack/Desktop/YOUTU

SystemExit: 1

In [5]:
!sudo lsof -i :34095

COMMAND      PID USER   FD   TYPE   DEVICE SIZE/OFF NODE NAME
jupyter-n 431792 jack   32u  IPv4 10319646      0t0  TCP localhost:36568->localhost:34095 (ESTABLISHED)
python    432267 jack   10u  IPv4 10319627      0t0  TCP localhost:34095 (LISTEN)
python    432267 jack   17u  IPv4 10318741      0t0  TCP localhost:36542->localhost:34095 (ESTABLISHED)
python    432267 jack   18u  IPv4 10318742      0t0  TCP localhost:34095->localhost:36542 (ESTABLISHED)
python    432267 jack   52u  IPv4 10319652      0t0  TCP localhost:34095->localhost:36568 (ESTABLISHED)


In [ ]:
kill <PID>

In [3]:
!netstat -tuln | grep 34095
tcp        0      0 127.0.0.1:34095         0.0.0.0:*               LISTEN     

tcp        0      0 127.0.0.1:34095         0.0.0.0:*               LISTEN     


In [ ]:
from IPython.display import IFrame

app_url = 'http://127.0.0.1:5000'  # Adjust the URL based on your Flask app configuration
IFrame(app_url, width=800, height=600)


In [22]:
from pydub import AudioSegment

def deepen_and_roughen_voice(input_file, output_file, pitch_shift, speed_factor):
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Shift the pitch (lower values make the voice deeper)
    pitched_audio = audio.speedup(playback_speed=speed_factor)
    pitched_audio = pitched_audio.set_frame_rate(audio.frame_rate)

    # Save the modified audio to the output file
    pitched_audio.export(output_file, format="wav")

if __name__ == "__main__":
    input_audio_file = "input_audio.wav"  # Replace with your input audio file
    output_audio_file = "output_audio.wav"  # Replace with the desired output file name

    deepen_and_roughen_voice(input_audio_file, output_audio_file, pitch_shift=-1.5, speed_factor=1.2)


In [23]:
from IPython.display import Audio

audio_file_path = "output_audio.wav"  # Replace with the path to your .wav file
Audio(audio_file_path)

In [10]:
!ls -rant static/audio_mp3

total 744
-rw-rw-r-- 1 1000 1000  32448 Aug 11 08:42  When-you-have-a-working-a.mp3
-rw-rw-r-- 1 1000 1000  45024 Aug 11 08:44  In-the-read_text.html-I-w.mp3
-rw-rw-r-- 1 1000 1000   8352 Aug 11 08:53  Now-is-time-for-a-test..mp3
-rw-rw-r-- 1 1000 1000  17184 Aug 11 08:57  We-are-still-texting-this.mp3
-rw-rw-r-- 1 1000 1000  36960 Aug 11 09:08  I-will-be-uploading-the-e.mp3
-rw-rw-r-- 1 1000 1000  20544 Aug 11 09:15  This-is-near-the-end-of-t.mp3
-rw-rw-r-- 1 1000 1000  35136 Aug 11 09:16  This-makes-the-basic-Flas.mp3
-rw-rw-r-- 1 1000 1000  24768 Aug 11 09:18  If-you-find-this-video-en.mp3
drwxrwxr-x 9 1000 1000   4096 Aug 11 10:03  ..
-rw-rw-r-- 1 1000 1000  22368 Aug 11 10:19  Lets-Start-with-a-a-littl.mp3
-rw-rw-r-- 1 1000 1000  11424 Aug 11 10:21  Notice-I-have-a-favico-ic.mp3
-rw-rw-r-- 1 1000 1000  11904 Aug 11 10:22  Now-we-have-centered-the-.mp3
-rw-rw-r-- 1 1000 1000   8736 Aug 11 10:27  Time-for-a-background-col.mp3
-rw-rw-r-- 1 1000 1000   7584 Aug 11 10:28  It-is-a-bit-p

In [11]:
!ffmpeg -i static/audio_mp3/I-need-an-audio-file-I-ca.mp3 input_audio.wav

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e